In [2]:
%load_ext autoreload
%autoreload 2

In [16]:
import pandas as pd
import pybaseball
from data_loader import PitchDataLoader
import processing as prc
import stuff_model as stuff
import joblib

import pandas as pd
import numpy as np
import skfuzzy as fuzz
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity
from joblib import Parallel, delayed
from xgboost import XGBRegressor
from tune_xgboost import tune_xgboost, distill_params
from typing import List, Dict, Tuple, Optional

In [4]:
path = 'data/'
pybaseball.cache.enable()

In [5]:
pdl = PitchDataLoader(path=path, start_date='2016-03-01', end_date='2024-11-15')
#pdl.load_new_data()

In [5]:
df = pdl.df.copy()
df = prc.apply_all_filters(df, prc.default_filters()).copy()
df = prc.save_memory(df, cols_to_drop=['des']).copy()
df = prc.calculate_new_features(df).copy()
df = prc.save_memory(df).copy()
df.to_parquet(f'{path}mem_eff_pitch_data.parquet')

In [6]:
df = pd.read_parquet(f'{path}mem_eff_pitch_data.parquet')

In [23]:
stuff_features = ['speed', 'speed_diff', 'lift', 'lift_diff', 'transverse_pit', 'transverse_pit_diff', 
                  'release_pos_x_pit', 'release_pos_y', 'release_pos_z', 'vert_approach_angle_adj']
cluster_target = 'csw'

df = stuff.assign_fuzzy_clusters(df, stuff_features, cluster_target)
df = prc.save_memory(df)
df.to_parquet(f'{path}clustered_pitch_data.parquet')

100%|██████████| 30/30 [01:57<00:00,  3.91s/trial, best loss: 0.59476561477669]  
{'cluster1': 'gyro slider', 'cluster2': 'cutter', 'cluster3': 'offspeed', 'cluster4': 'high-slot fastball', 'cluster5': 'sweeper', 'cluster6': 'low-slot fastball', 'cluster7': 'curveball', 'cluster8': 'sinker'}


In [24]:
cluster_dist = stuff.create_location_distributions(df)
count_frequencies = stuff.calculate_count_frequencies(df)
platoon_cluster_dist = stuff.combine_flatten_distributions(cluster_dist, count_frequencies)
joblib.dump(platoon_cluster_dist, 'data/platoon_cluster_dist.dat')

c:\Users\calal\PitchSim\stuff_model.py:170: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df[cols].groupby(group_cols)


['data/platoon_cluster_dist.dat']

In [25]:
feat = ['speed', 'vert_approach_angle_adj', 'transverse', 'transverse_pit', 'transverse_bat', 
        'lift', 'release_pos_x', 'release_pos_x_pit','release_pos_x_bat', 'release_pos_y', 
        'release_pos_z', 'plate_x', 'plate_x_pit','plate_x_bat', 'plate_x_abs', 'plate_z', 
        'plate_z_top', 'plate_z_bot', 'plate_dist', 'balls', 'strikes', 'speed_diff', 
        'lift_diff', 'transverse_pit_diff', 'vert_approach_angle', 'game_year']

model_filters = stuff.get_model_filters(df)
xgb_models = stuff.train_models(df, model_filters, feat)
joblib.dump(xgb_models, 'data/xgb_models.dat')

100%|██████████| 30/30 [00:42<00:00,  1.42s/trial, best loss: 0.2842531978844976] 


['data/xgb_models.dat']

In [14]:
df = pd.read_parquet(f'{path}clustered_pitch_data.parquet')

In [10]:
batch_size = 5000
n_batches = 3
rv = pd.read_csv('runvalue.csv')
dist = joblib.load('data/platoon_cluster_dist.dat')
features = ['speed', 'vert_approach_angle_adj', 'transverse', 'transverse_pit', 'transverse_bat', 
        'lift', 'release_pos_x', 'release_pos_x_pit','release_pos_x_bat', 'release_pos_y', 
        'release_pos_z', 'plate_x', 'plate_x_pit','plate_x_bat', 'plate_x_abs', 'plate_z', 
        'plate_z_top', 'plate_z_bot', 'plate_dist', 'balls', 'strikes', 'speed_diff', 
        'lift_diff', 'transverse_pit_diff', 'vert_approach_angle', 'game_year']
xgb_models = joblib.load('data/xgb_models.dat')
path = 'data/'
#sim = stuff.simulate_pitches(df, batch_size, n_batches, dist, features, rv, xgb_models, path)

In [11]:
simR = []
simL = []
for i in range(n_batches):
    simR.append(pd.read_parquet(f'data/sim_vsR_batch{i+1}.parquet'))
    simL.append(pd.read_parquet(f'data/sim_vsL_batch{i+1}.parquet'))
simR = pd.concat(simR).reset_index(drop=True)
simL = pd.concat(simL).reset_index(drop=True)
simR.to_parquet(f'{path}sim_vsR.parquet')
simL.to_parquet(f'{path}sim_vsL.parquet')

In [12]:
distill_features = ['speed', 'speed_diff', 'lift', 'lift_diff', 
                    'transverse', 'transverse_pit', 'transverse_pit_diff', 
                    'release_pos_x', 'release_pos_x_pit', 'release_pos_y', 'release_pos_z', 
                    'vert_approach_angle_adj']
events = [x for x in simR.columns if x.startswith('x_')]
vsR_models = stuff.train_distilled_models(simR, distill_features, events)
vsL_models = stuff.train_distilled_models(simL, distill_features, events)

df = stuff.make_distilled_predictions(df, distill_features, events, vsR_models, vsL_models)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

c:\Users\calal\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:59:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)

c:\Users\calal\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:59:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_

100%|██████████| 10/10 [00:40<00:00,  4.00s/trial, best loss: 3.152913242729707e-06]
x_callstr
x_ball
x_hbp
x_swstr
x_foul
x_pop
x_hr
x_if1b
x_gbout
x_gidp
x_air_out
x_1b
x_2b
x_3b
x_run_value


In [17]:
df = stuff.make_distilled_predictions(df, distill_features, events, vsR_models, vsL_models)
stuff.generate_results_csv(df, distill_features)

x_callstr
x_ball
x_hbp
x_swstr
x_foul
x_pop
x_hr
x_if1b
x_gbout
x_gidp
x_air_out
x_1b
x_2b
x_3b
x_run_value


c:\Users\calal\PitchSim\stuff_model.py:525: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp_filt = df[cols].groupby(by=group_cols).filter(lambda x: len(x) >= 10)
c:\Users\calal\PitchSim\stuff_model.py:528: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  results = grp_filt[cols].groupby(by=group_cols).mean().dropna().reset_index()
c:\Users\calal\PitchSim\stuff_model.py:529: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  re